In [63]:
# For data manipulation
import pandas as pd         

# Garbage Collector to free up memory
import gc                         
gc.enable()                       

In [64]:
orders = pd.read_csv('orders.csv' )
order_products_train = pd.read_csv('order_products__train.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')
products = pd.read_csv('products.csv')
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')

In [65]:
#Merge the orders DF with order_products_prior by their order_id, keep only these rows with order_id that they are appear on both DFs
op = orders.merge(order_products_prior, on='order_id', how='inner') ##
op.head()


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [66]:
##User Predictors
#1.Number of orders per customer
#2.How frequent a customer has reordered products

In [67]:
##1.Number of orders per customer
# Create distinct groups for each user, identify the highest order number in each group, save the new column to a DataFrame
user = op.groupby('user_id')['order_number'].max().to_frame('u_total_orders')
user.head()
# Reset the index of the DF so to bring user_id from index to column (pre-requisite for step 2.4)
user = user.reset_index()
user.head()

,user_id,u_total_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [68]:
#2.How frequent a customer has reordered products
u_reorder = op.groupby('user_id')['reordered'].mean().to_frame('u_reordered_ratio') #
u_reorder = u_reorder.reset_index()
u_reorder.head()

,user_id,u_reordered_ratio
0,1,0.694915
1,2,0.476923
2,3,0.625000
3,4,0.055556
4,5,0.378378


In [69]:
##User Predictors
user = user.merge(u_reorder, on='user_id', how='left') #

del u_reorder
gc.collect()

user.head()

,user_id,u_total_orders,u_reordered_ratio
0,1,10,0.694915
1,2,14,0.476923
2,3,12,0.625000
3,4,5,0.055556
4,5,4,0.378378


In [70]:
##Product Predictors
#1.Number of purchases for each product
#2.What is the probability for a product to be reordered

In [71]:
##1.Number of purchases for each product
# Create distinct groups for each product, count the orders, save the result for each product to a new DataFrame  
prd = op.groupby('product_id')['order_id'].count().to_frame('p_total_purchases')
prd.head()
# Reset the index of the DF so to bring product_id rom index to column
prd = prd.reset_index()
prd.head()

,product_id,p_total_purchases
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [72]:
##2.What is the probability for a product to be reordered
#Remove prodcuts with less than 40 orders.
p_reorder = op.groupby('product_id').filter(lambda x: x.shape[0] >40)
p_reorder.head()

#Calculate the mean of redordes
p_reorder = p_reorder.groupby('product_id')['reordered'].mean().to_frame('p_reorder_ratio')
p_reorder = p_reorder.reset_index()
p_reorder.head()

,product_id,p_reorder_ratio
0,1,0.613391
1,2,0.133333
2,3,0.732852
3,4,0.446809
4,8,0.503030


In [73]:
##Product Predictors
#Merge the prd DataFrame with reorder
prd = prd.merge(p_reorder, on='product_id', how='left')

#delete the reorder DataFrame
del p_reorder
gc.collect()

prd.head()

,product_id,p_total_purchases,p_reorder_ratio
0,1,1852,0.613391
1,2,90,0.133333
2,3,277,0.732852
3,4,329,0.446809
4,5,15,NaN


In [74]:
sum(prd.p_reorder_ratio.isna())

21001

In [75]:
#Fill the NA that caused by filtering out 40 less orders 
prd['p_reorder_ratio'] = prd['p_reorder_ratio'].fillna(0) #
prd.head()

,product_id,p_total_purchases,p_reorder_ratio
0,1,1852,0.613391
1,2,90,0.133333
2,3,277,0.732852
3,4,329,0.446809
4,5,15,0.000000


In [76]:
##Create user-product predictors
#1.How many times a user bought a product
#2.How frequently a customer bought a product after its first purchase
#3.How many times a customer bought a product in its last 5 orders

In [77]:
##1.How many times a user bought a product
# Create distinct groups for each combination of user and product, count orders, save the result for each user X product to a new DataFrame 
uxp = op.groupby(['user_id', 'product_id'])['order_id'].count().to_frame('uxp_total_bought')
uxp = uxp.reset_index()
uxp.head()

,user_id,product_id,uxp_total_bought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [78]:
##2.How frequently a customer bought a product after its first purchase
#probability a customer buy a product after the product's first purchase = times a user bought a product / number of orders since the product's first order

In [79]:
#times a user bought a product
times = op.groupby(['user_id', 'product_id'])[['order_id']].count()
times.columns = ['Times_Bought_N']
times.head()

Times_Bought_N
user_id product_id                
1       196                     10
        10258                    9
        10326                    1
        12427                   10
        13032                    3

In [80]:
#number of orders since the product's first order
total_orders = op.groupby('user_id')['order_number'].max().to_frame('total_orders') 


first_order_no = op.groupby(['user_id', 'product_id'])['order_number'].min().to_frame('first_order_number')
first_order_no  = first_order_no.reset_index()

span = pd.merge(total_orders, first_order_no, on='user_id', how='right')

# The +1 includes in the difference the first order were the product has been purchased
span['Order_Range_D'] = span.total_orders - span.first_order_number + 1

span.head()

,user_id,total_orders,product_id,first_order_number,Order_Range_D
0,1,10,196,1,10
1,1,10,10258,2,9
2,1,10,10326,5,6
3,1,10,12427,1,10
4,1,10,13032,2,9


In [81]:
##2.How frequently a customer bought a product after its first purchase
uxp_ratio = pd.merge(times, span, on=['user_id', 'product_id'], how='left')
uxp_ratio['uxp_reorder_ratio'] = uxp_ratio.Times_Bought_N / uxp_ratio.Order_Range_D 
uxp_ratio = uxp_ratio.drop(['Times_Bought_N', 'total_orders', 'first_order_number', 'Order_Range_D'], axis=1)
uxp_ratio.head()

,user_id,product_id,uxp_reorder_ratio
0,1,196,1.000000
1,1,10258,1.000000
2,1,10326,0.166667
3,1,12427,1.000000
4,1,13032,0.333333


In [82]:
#Remove temporary DataFrames
del [times, first_order_no, span]

In [83]:
uxp = uxp.merge(uxp_ratio, on=['user_id', 'product_id'], how='left')

del uxp_ratio
uxp.head()

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio
0,1,196,10,1.000000
1,1,10258,9,1.000000
2,1,10326,1,0.166667
3,1,12427,10,1.000000
4,1,13032,3,0.333333


In [84]:
##3.How many times a customer bought a product on its last 5 orders
#probability a customer buy a product in the last 5 orders = times a customer bought the product in last 5 orders / 5
op['order_number_back'] = op.groupby('user_id')['order_number'].transform(max) - op.order_number +1 
op5 = op[op.order_number_back <= 5]

last_five = op5.groupby(['user_id','product_id'])[['order_id']].count()
last_five.columns = ['times_last5']


,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,times_last5
0,1,196,10,1.000000,5.0
1,1,10258,9,1.000000,5.0
2,1,10326,1,0.166667,0.0
3,1,12427,10,1.000000,5.0
4,1,13032,3,0.333333,2.0


In [ ]:
##user-product predictors
uxp = uxp.merge(last_five, on=['user_id', 'product_id'], how='left')

del [op5 , last_five]
uxp = uxp.fillna(0)
uxp.head()

In [85]:
#Merge uxp features with the user features
#Store the results on a new DataFrame
data = uxp.merge(user, on='user_id', how='left')
data.head()

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio
0,1,196,10,1.000000,5.0,10,0.694915
1,1,10258,9,1.000000,5.0,10,0.694915
2,1,10326,1,0.166667,0.0,10,0.694915
3,1,12427,10,1.000000,5.0,10,0.694915
4,1,13032,3,0.333333,2.0,10,0.694915


In [86]:
#Merge uxp & user features (the new DataFrame) with prd features
data = data.merge(prd, on='product_id', how='left')
data.head()

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio,p_total_purchases,p_reorder_ratio
0,1,196,10,1.000000,5.0,10,0.694915,35791,0.776480
1,1,10258,9,1.000000,5.0,10,0.694915,1946,0.713772
2,1,10326,1,0.166667,0.0,10,0.694915,5526,0.652009
3,1,12427,10,1.000000,5.0,10,0.694915,6476,0.740735
4,1,13032,3,0.333333,2.0,10,0.694915,3751,0.657158


In [87]:
del op, user, prd, uxp
gc.collect()

171

In [88]:
##Create train and test dataframes
# In one step exclude all the prior orders so to deal with the future orders from all customers
orders_future = orders.loc[orders.eval_set!='prior', ['user_id', 'eval_set', 'order_id'] ]
orders_future.head(10)

,user_id,eval_set,order_id
10,1,train,1187899
25,2,train,1492625
38,3,test,2774568
44,4,test,329954
49,5,train,2196797
53,6,test,1528013
74,7,train,525192
78,8,train,880375
82,9,train,1094988
88,10,train,1822501


In [89]:
# bring the info of the future orders to data DF
data = data.merge(orders_future, on='user_id', how='left')
data.head(20)

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio,p_total_purchases,p_reorder_ratio,eval_set,order_id
0,1,196,10,1.000000,5.0,10,0.694915,35791,0.776480,train,1187899
1,1,10258,9,1.000000,5.0,10,0.694915,1946,0.713772,train,1187899
2,1,10326,1,0.166667,0.0,10,0.694915,5526,0.652009,train,1187899
3,1,12427,10,1.000000,5.0,10,0.694915,6476,0.740735,train,1187899
4,1,13032,3,0.333333,2.0,10,0.694915,3751,0.657158,train,1187899
5,1,13176,2,0.222222,0.0,10,0.694915,379450,0.832555,train,1187899
6,1,14084,1,0.100000,0.0,10,0.694915,15935,0.810982,train,1187899
7,1,17122,1,0.166667,0.0,10,0.694915,13880,0.675576,train,1187899
8,1,25133,8,1.000000,5.0,10,0.694915,6196,0.740155,train,1187899
9,1,26088,2,0.200000,0.0,10,0.694915,2523,0.539041,train,1187899


In [90]:
#Keep only the customers who we know what they bought in their future order (train dataframe)
data_train = data[data.eval_set=='train']
data_train.head()

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio,p_total_purchases,p_reorder_ratio,eval_set,order_id
0,1,196,10,1.000000,5.0,10,0.694915,35791,0.776480,train,1187899
1,1,10258,9,1.000000,5.0,10,0.694915,1946,0.713772,train,1187899
2,1,10326,1,0.166667,0.0,10,0.694915,5526,0.652009,train,1187899
3,1,12427,10,1.000000,5.0,10,0.694915,6476,0.740735,train,1187899
4,1,13032,3,0.333333,2.0,10,0.694915,3751,0.657158,train,1187899


In [91]:
#Get from order_products_train all the products that the train users whether reorder in their future order
data_train = data_train.merge(order_products_train[['product_id','order_id', 'reordered']], on=['product_id','order_id'], how='left' )
data_train.head()

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio,p_total_purchases,p_reorder_ratio,eval_set,order_id,reordered
0,1,196,10,1.000000,5.0,10,0.694915,35791,0.776480,train,1187899,1.0
1,1,10258,9,1.000000,5.0,10,0.694915,1946,0.713772,train,1187899,1.0
2,1,10326,1,0.166667,0.0,10,0.694915,5526,0.652009,train,1187899,NaN
3,1,12427,10,1.000000,5.0,10,0.694915,6476,0.740735,train,1187899,NaN
4,1,13032,3,0.333333,2.0,10,0.694915,3751,0.657158,train,1187899,1.0


In [92]:
data_train['reordered'] = data_train['reordered'].fillna(0)
data_train.head()

,user_id,product_id,uxp_total_bought,uxp_reorder_ratio,times_last5,u_total_orders,u_reordered_ratio,p_total_purchases,p_reorder_ratio,eval_set,order_id,reordered
0,1,196,10,1.000000,5.0,10,0.694915,35791,0.776480,train,1187899,1.0
1,1,10258,9,1.000000,5.0,10,0.694915,1946,0.713772,train,1187899,1.0
2,1,10326,1,0.166667,0.0,10,0.694915,5526,0.652009,train,1187899,0.0
3,1,12427,10,1.000000,5.0,10,0.694915,6476,0.740735,train,1187899,0.0
4,1,13032,3,0.333333,2.0,10,0.694915,3751,0.657158,train,1187899,1.0


In [93]:
#set user_id and product_id as the index of the data_train
data_train = data_train.set_index(['user_id', 'product_id'])
data_train.head(15)

uxp_total_bought  uxp_reorder_ratio  times_last5  \
user_id product_id                                                     
1       196                       10           1.000000          5.0   
        10258                      9           1.000000          5.0   
        10326                      1           0.166667          0.0   
        12427                     10           1.000000          5.0   
        13032                      3           0.333333          2.0   
        13176                      2           0.222222          0.0   
        14084                      1           0.100000          0.0   
        17122                      1           0.166667          0.0   
        25133                      8           1.000000          5.0   
        26088                      2           0.200000          0.0   
        26405                      2           0.200000          0.0   
        30450                      1           0.125000          0.0   
        35951                      1           1.000000          1.0   
        38928                      1           1.000000          1.0   
        39657                      1           1.000000          1.0   

                    u_total_orders  u_reordered_ratio  p_total_purchases  \
user_id product_id                                                         
1       196                     10           0.694915              35791   
        10258                   10           0.694915               1946   
        10326                   10           0.694915               5526   
        12427                   10           0.694915               6476   
        13032                   10           0.694915               3751   
        13176                   10           0.694915             379450   
        14084                   10           0.694915              15935   
        17122                   10           0.694915              13880   
        25133                   10           0.694915               6196   
        26088                   10           0.694915               2523   
        26405                   10           0.694915               1214   
        30450                   10           0.694915              21490   
        35951                   10           0.694915              57895   
        38928                   10           0.694915              13238   
        39657                   10           0.694915               5019   

                    p_reorder_ratio eval_set  order_id  reordered  
user_id product_id                                                 
1       196                0.776480    train   1187899        1.0  
        10258              0.713772    train   1187899        1.0  
        10326              0.652009    train   1187899        0.0  
        12427              0.740735    train   1187899        0.0  
        13032              0.657158    train   1187899        1.0  
        13176              0.832555    train   1187899        0.0  
        14084              0.810982    train   1187899        0.0  
        17122              0.675576    train   1187899        0.0  
        25133              0.740155    train   1187899        1.0  
        26088              0.539041    train   1187899        1.0  
        26405              0.441516    train   1187899        1.0  
        30450              0.566682    train   1187899        0.0  
        35951              0.757837    train   1187899        0.0  
        38928              0.827769    train   1187899        1.0  
        39657              0.766288    train   1187899        1.0

In [94]:
#Remove all non-predictor variables
data_train = data_train.drop(['eval_set', 'order_id'], axis=1)
data_train.head(15)

uxp_total_bought  uxp_reorder_ratio  times_last5  \
user_id product_id                                                     
1       196                       10           1.000000          5.0   
        10258                      9           1.000000          5.0   
        10326                      1           0.166667          0.0   
        12427                     10           1.000000          5.0   
        13032                      3           0.333333          2.0   
        13176                      2           0.222222          0.0   
        14084                      1           0.100000          0.0   
        17122                      1           0.166667          0.0   
        25133                      8           1.000000          5.0   
        26088                      2           0.200000          0.0   
        26405                      2           0.200000          0.0   
        30450                      1           0.125000          0.0   
        35951                      1           1.000000          1.0   
        38928                      1           1.000000          1.0   
        39657                      1           1.000000          1.0   

                    u_total_orders  u_reordered_ratio  p_total_purchases  \
user_id product_id                                                         
1       196                     10           0.694915              35791   
        10258                   10           0.694915               1946   
        10326                   10           0.694915               5526   
        12427                   10           0.694915               6476   
        13032                   10           0.694915               3751   
        13176                   10           0.694915             379450   
        14084                   10           0.694915              15935   
        17122                   10           0.694915              13880   
        25133                   10           0.694915               6196   
        26088                   10           0.694915               2523   
        26405                   10           0.694915               1214   
        30450                   10           0.694915              21490   
        35951                   10           0.694915              57895   
        38928                   10           0.694915              13238   
        39657                   10           0.694915               5019   

                    p_reorder_ratio  reordered  
user_id product_id                              
1       196                0.776480        1.0  
        10258              0.713772        1.0  
        10326              0.652009        0.0  
        12427              0.740735        0.0  
        13032              0.657158        1.0  
        13176              0.832555        0.0  
        14084              0.810982        0.0  
        17122              0.675576        0.0  
        25133              0.740155        1.0  
        26088              0.539041        1.0  
        26405              0.441516        1.0  
        30450              0.566682        0.0  
        35951              0.757837        0.0  
        38928              0.827769        1.0  
        39657              0.766288        1.0

In [95]:
#data_train.to_csv('data_train.csv', index=False)